In [1]:
import json
import numpy as np
import pandas as pd
import urllib.request
import folium
from folium.plugins import MarkerCluster
from datetime import datetime
import plotly.express as px

In [14]:
def get_latest_earthquake_view():
    """
    Display the latest earthquake map
    """
    # download the earthquake data
    with urllib.request.urlopen('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_hour.geojson') as url:
        data = json.loads(url.read().decode())
        
    # Get the core data from the JSON.
    features = data['features']
    df = pd.DataFrame()
    lat_list = []
    long_list = []
    mag_list = []
    place_list = []
    time_list = []
    
    for earthquake in features:
        long_list.append(earthquake['geometry']['coordinates'][0])
        lat_list.append(earthquake['geometry']['coordinates'][1])
        mag_list.append(earthquake['properties']['mag'])
        place_list.append(earthquake['properties']['place'])
        time_list.append(earthquake['properties']['time'])
        
    df['place'] = place_list
    df['long'] = long_list
    df['lat'] = lat_list
    df['mag'] = mag_list
    df['time'] = time_list
    
    # Create an empty map first
    us_center = [37.09, -95.72]
    themap = folium.Map(location=us_center, zoom_start=2)
    
    for index, row in df.iterrows():
        NL = '<br/>'
        
        info = 'Location: ' + row['place'] + NL +        \
               'Mag: ' + str(row['mag']) + NL +          \
               'Lat: ' + str(row['lat']) + NL +          \
               'Long: ' + str(row['long']) + NL +        \
               'Datetime: ' + str(datetime.fromtimestamp(row['time'] / 1000))
        
        popup_iframe = folium.IFrame(info,
                                    width=200,
                                    height=150)
        popup_info = folium.Popup(popup_iframe, width=100)
        folium.Marker([row['lat'], row['long']], popup=popup_info).add_to(themap)
    
    title_html = '''
    <h3 align="center" style="font-size:20px"><b>Latest Earthquake Map</b></h3>
    '''
    # Add a title to the map
    themap.get_root().html.add_child(folium.Element(title_html))
    display(themap)
    
    # histogram
    fig = px.histogram(df, x='mag')
    fig.show()

In [15]:
get_latest_earthquake_view()